In [1]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import numpy as npZ

from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [7]:
# Load Model
# Load the model architecture
vgg16 = models.vgg16(pretrained=False)
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # VGG16's original FC layer
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),   # VGG16's original FC layer
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 7)      # New output layer for 9 classes
)

In [ ]:
# Load the fine-tuned weights
vgg16.load_state_dict(torch.load('../vgg16_finetuned_pytorch.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the image
# image = Image.open("../../flask/images-cropped/2025-02-22 15_34_43webcam_cropped_0.png")
# image = Image.open("../../flask/images-cropped/2025-02-22 15_36_01webcam_cropped_0.png")
# image = Image.open("../../flask/images-cropped/2025-02-22 15_37_36webcam_cropped_0.png")
image = Image.open("../../flask/images-cropped/2025-02-22 15_38_30webcam_cropped_0.png")
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

# Run inference
with torch.no_grad():
    output = vgg16(input_batch)

# Convert output to probabilities (optional)
probabilities = F.softmax(output, dim=1)

# Get the predicted class
_, predicted_class = torch.max(output, 1)
predicted_class = predicted_class.item()

# Map the predicted class to a label
class_labels = ["class_0", "class_1", "class_2", "class_3", "class_4",
                "class_5", "class_6"]
predicted_label = class_labels[predicted_class]

# Print the result
print(f"Predicted class: {predicted_label}")
print(f"Probabilities: {probabilities}")

Predicted class: class_1
Probabilities: tensor([[5.2916e-03, 9.6843e-01, 1.0987e-02, 1.1683e-05, 6.5149e-04, 1.4619e-02,
         1.2202e-05]])
